# Calculating Bond Payment Dates And Amounts

## Background Material

The examples and discussions in this chapter utilize the **datetime** and **holidays** Python libraries.

* The **datetime** library is introduced in the *Manipulating Dates* notebook of Chapter Two, and is further described in the introductory volume, **Background Material: An Introduction to Python for Financial Python**.  
  * A quick reference is available in: "[A Quick Introduction to Manipulating Dates](https://patrickjhess.github.io/Introduction-To-Python-For-Financial-Python/An_Introduction_To_NumPy.html)".  
  * The complete introductory volume is located at: "[Background Material: An Introduction to Python for Financial Python](https://patrickjhess.github.io/Introduction-To-Python-For-Financial-Python/intro.html)".  
  * Consult these cited materials for a more detailed discussion of Python concepts.  
* The **holidays** library, along with the `easter` function and determining weekends, is demonstrated in the notebook, *Accounting For Non Settlement Days: Holidays, Weekends, And Good Friday*.

## Importing Libraries, Modules, And Functions

As in Chapter One of the volume, modules that are included in the standard Python library are imported. When necessary, other modules or libraries are installed before they are imported. If necessary, The new module <font color='green'>holidays</font> is installed and then imported.  

```
import sys
import requests
from datetime import datetime, date, timedelta
from types import ModuleType
import calendar

try:
    from IPython.display import Markdown, display
except:
    !pip -q install IPython
    from IPython.display import Markdown, displayr

try:
    import numpy as np
except:
    !pip -q install numpy
    import numpy as np

try:
    from dateutil.relativedelta import relativedelta
except:
    !pip install python-dateutil
    from dateutil.relativedelta import relativedelta
    from datetutil.easter import easter

try:
    import holidays
except:
    !pip -q install holidays
    import holidays
```

In [ ]:
# Import OS to interact with local computer operating system
import sys
import requests
from types import ModuleType
# Import the datetime and date classes from the datetime module, and timedelta for working with dates.
from datetime import datetime, date, timedelta
#Import markdown and display
from IPython.display import Markdown as md, display
# Last calendar day of the month and day of the week for first day
import calendar

# Ipython not builtin, install if  necessary with quiet flag (-q)
try:
    from IPython.display import Markdown, display
except:
    !pip -q install IPython
    from IPython.display import Markdown, displayr

# Pandas not builtin, install if  necessary with quiet flag (-q)
try:
    import pandas as pd
except:
    !pip -q install pandas
    import pandas as pd

# NumPy not builtin, install if  necessary with quiet flag (-q)
try:
    import numpy as np
except:
    !pip -q install numpy
    import numpy as np

# dateutilnot builtin, install if  necessary with quiet flag (-q)
try:
    from dateutil.relativedelta import relativedelta
except:
    !pip -q install python-dateutil
    from dateutil.relativedelta import relativedelta
    from dateutil.easter import easter

## Adding A Custom Module And Importing Functions

Like Chapter One the custom module <font color='green'>module_basic_concepts_fixed_income</font> custom module is accessed from Dropbox and named <font color='green'>basic_concepts_fixed_income</font>. Two functions are imported. The first is the function <font color='green'>last_day_month()</font> that was developed in the previous chapter. The second function is <font color='green'>bond_pay_data()</font> and is developed in this chapter. developed in this

 and is a helper function a new function  That function, however, relies upon a number of helper functions developed in Chapter Two and the vectorized function <font color='green'>adjust_bond_pay_dates()</font> developed in this chapter.

```
from basic_income_module import (bond_pay_data)
```

*   <font color='green'>bond_pay_data()</font> (This Chapter) Returns NumPy arrays of payment dates and amounts.<a href='https://docs.google.com/document/d/e/2PACX-1vQGqmvajJO28fLjbClysLCUTxH-TvNII2waYX1eYO2c8Gbra-YkVy3OmBQucKJj8LM5PzhDaJBBfGSj/pub'>View Here</a>.



In [ ]:
import requests, sys
from types import ModuleType
# Define the URL of the Python module to be downloaded from Dropbox.
# The 'dl=1' parameter in the URL forces a direct download of the file content.
url= 'https://www.dropbox.com/scl/fi/4y5hjxlfphh1ngvbgo77q/\
module_-basic_concepts_fixed_income.py?rlkey=6oxi7mgka42veaat79hcv8boz&st=87sztshr&dl=1'
module_name='basic_concepts_fixed_income'
# Send an HTTP GET request to the URL and store the server's response.
try:
  response=requests.get(url)
  # Raise an exception for bad status codes (like 404 Not Found)
  response.raise_for_status()
  module= ModuleType(module_name)
  #Code contained in response.text executed
  exec(response.text, module.__dict__)
  # Module added to sys
  sys.modules[module_name]=module
except requests.exceptions.RequestException as e:
    print(f"❌ Error: Could not fetch module from URL. {e}")
except Exception as e:
    print(f"❌ Error: Failed to execute or import the module. {e}")

# Now that 'basic_concepts_fixed_income' exists in the notebook, import the specific functions
from basic_concepts_fixed_income import (validate_date,
                                         last_day_month,
                                         adjust_bond_pay_dates)

## Scheduled Pay Dates
Because a bond's maturity is a scheduled pay date, all other scheduled pay dates can be calculated from that date by stepping backwards by the time between payments.  The heart of the function is six statements. The <font color='green'>dates</font> list is created and <font color='green'>current_date</font> is assigned the <font color='green'>maturity</font> date.  At each iteration of the <font color='green'>while</font> loop a value is appended to the <font color='green'>dates</font> list and the value of <font color='green'>current_date</font> is decreased by the time between payments, <font color='green'>num_months</font>.  For a semi-annual bond, <font color='green'>num_months</font> is six. As long as the value of <font color='green'>current_date</font> is greater than the <font color='green'>settlement</font> date, dates are appended to <font color='green'>dates</font> list.



```
 dates=[]
 current_date=maturity
 # Loop backward from the maturity date.
 while current_date>settlement:
   if end_month:current_date=last_day_month(current_date)
   dates.append(current_date)
   current_date -= relativedelta(months=num_months)
```
Because the function starts at the maturity date, the payument dates are in reverse chronological order.  The chronological ordered dates are returned by slicing the dates list with an increment of -1.$^{1}$


```
return dates[::-1]
```


---
$^{1} see [see A Quick Introduction To Lists](https://)



In [ ]:
def scheduled_pay_dates(maturity,settlement,freq):
  '''

  '''
  from datetime import datetime,date, timedelta
  from dateutil.relativedelta import relativedelta
  import pandas as pd

  #Validate the data- maturity, coupon, settlement, freq
  #maturity
  maturity=validate_date(maturity)

  #settlement
  if settlement is None:
      settlement = date.today()
  else:
      settlement=validate_date(settlement)
  #freq
  if int(freq) not in [1,2,4,12]:
      display(md(f"### ⚠️  your assigned freq {freq} it must be (1, 2, 4, or 12)\
     \n     semi-annual assumed (2)."))
      freq=int(2)

  # Calculate the number of months between each coupon payment.
  num_months=int(12/freq)

  #Need to check for month_end
  end_month=pd.Timestamp(maturity).is_month_end

  # This list will store the scheduled payment dates.
  dates=[]
  current_date=maturity

  # Loop backward from the maturity date.
  while current_date>settlement:
    if end_month:current_date=last_day_month(current_date)
    dates.append(current_date)
    current_date -= relativedelta(months=num_months)

  # The dates were generated backward, slice to for chronological order
  return dates[::-1]

 The <font color='green'>bond_pay_data()</font> function requires two arguments: <font color='green'>maturity</font> (the maturity date) and <font color='green'>coupon</font> (the coupon). <font color='green'>settlement</font> date defaults to the current day. The payment frequency <font color='green'>freq</font>  defaults to semi-annual (2) but can be set to 1 (annual), 4 (quarterly), or 12 (monthly). Both <font color='green'>settlement</font> and <font color='green'>maturity</font> dates must be `datetime` or date objects.



If <font color='green'>coupon</font> is zero, a NumPy array with the maturity date and a NumPy array with the par value of 100 is returned. Otherwise, the NumPy array <font color='green'>pay</font> is created with length equal to the length of <font color='green'>dates</font> and a value equal to the annual coupon divided by the frequency of payments. The last value of <font color='green'>pay</font> is increased by the par value of 100.


```
pay=np.full(len(dates),coupon/freq)
pay[-1]+=100
```

The  <font color='green'>dates</font> list is passed to the vectorized version of <font color='green'>adjust_pay_dates()</font> as a NumPy array. A NumPy array of payment dates and a NumPy array of payments are returned


```
pay_dates=vect_adjust_bond_pay_dates(np.array(dates))
return pay_dates,pay


In [ ]:
def bond_pay_data(maturity,coupon,settlement=None,freq=2):
  '''
  Function calculates payment Dates And Amounts.
  maturity is a datetime object and coupon is a real number.
  Required arguments are maturity and annual coupon.
  If provided, the value of settlement is a datetime object;
  otherwise defaults to date.today()
  freq defaults to semi-annual but accepts freq equal
  to 1 for annual, 2 for semi-annal, 4 for quarterly, and 12 for monthly.
  The function assumes a par value of 100.
  Returns Numpy arrays of dates and amounts.

   Raises:
      TypeError: If maturity or settlement are not datetime objects.
      ValueError: If inputs are not logically valid (e.g., negative coupon,
                  maturity before settlement).
  '''
  from datetime import datetime,date, timedelta
  from dateutil.relativedelta import relativedelta
  import pandas as pd
  import numpy as np
  from IPython.display import display, Markdown as md

  #Validate the data- maturity, coupon, settlement, freq
  #maturity
  maturity=validate_date(maturity)

  #coupon
  try:
      coupon = float(coupon)
      if coupon < 0:
          raise ValueError("coupon rate cannot be negative.")
  except (ValueError, TypeError):
      raise ValueError("coupon must be a valid number.")

  #settlement
  if settlement is None:
      settlement = date.today()
  else:
      settlement=validate_date(settlement)

  #freq
  if int(freq) not in [1,2,4,12]:
      display(md(f"### ⚠️  your assigned freq {freq} it must be (1, 2, 4, or 12)\
      \n  ###    semi-annual assumed (2)."))
      freq=int(2)

  if coupon==0:
    #Adjust maturity for nonsettlement day and return date and face value
    adjust_maturity=adjust_bond_pay_dates(maturity)
    return np.array([adjust_maturity]),np.array([100.0])

  #get scheduled payment dates from helper function scheduled_pay_dates
  dates=scheduled_pay_dates(maturity,settlement,freq)

  #pandas apply method to calculate adjusted pay dates
  #Convert dates to pandas series
  dates_series=pd.Series(dates)

  #Use pandas apply to vectorize calculations
  adjusted_dates=dates_series.apply(adjust_bond_pay_dates)

  #Convert pandas series to numpy array
  pay_dates=adjusted_dates.to_numpy()

  #calculate payments
  #coupon divided by freq at each date
  pay=np.full(len(dates),coupon/freq)

  #Add principal payment as last cash payment
  pay[-1]+=100

  return pay_dates,pay

## Bond Payment Dates
The function is demonstrated with the February 29$^{th}$ 2028 4.0 Coupon Bond.
The diagram shows the correct pattern of pay dates and payments of the bond.
 February 29$^{th}$ 2028 4.0 Coupon Bond Payment Dates And Amounts
<img src='https://docs.google.com/drawings/d/e/2PACX-1vSaXeujj_5-3TMLbGrVHdZ-RkxmRv4TTkPH5aQ-JCxDsObw5Mj6XyyayVsBIEvAM42rNakP-Zmrlmq_/pub?w=960&h=540'>

The <font color='green'>bond_pay_data()</font> function relies upon the helper function <font color='green'>last_day_monthh()</font> introduced in Chapter Two: Accrued Interest.  The function is included in the module <font color='green'>basic_conceepts_fixed_income</font>.  Although it is available to <font color='green'>bond_pay_data()</font> imported from the module, it is not available to the function in the previous cell of the notebook.  If you executed this cell, <font color='green'>last_day_monht()</font> can not be accessed by <font color='green'>bond_pay_data()</font>.  In this instance you need to import it from <font color='green'>basic_conceepts_fixed_income</font> before you use <font color='green'>bond_pay_data()</font>.



```
from basic_concepts_fixed_income import (last_day_month)
```
Alternatively, you can import  <font color='green'>bond_pay_data()</font> again from <font color='green'>basic_conceepts_fixed_income</font> to reassign the function.



```
from basic_concepts_fixed_income import (bond_pay_data)
```




In [ ]:
from basic_concepts_fixed_income import (bond_pay_data)
maturity=date(2028,2,29)
settlement=date(2025,1,21)
coupon=4
bond_pay_data(maturity,coupon,settlement=settlement)

(array([datetime.date(2025, 2, 28), datetime.date(2025, 9, 2),
        datetime.date(2026, 3, 2), datetime.date(2026, 8, 31),
        datetime.date(2027, 3, 1), datetime.date(2027, 8, 31),
        datetime.date(2028, 2, 29)], dtype=object),
 array([  2.,   2.,   2.,   2.,   2.,   2., 102.]))

## <span style="text-align: left; color:green; font-family: 'Franklin Gothic Medium', sans-serif; margin-top: 1.0em; margin-bottom: 0em; font-style: italic;">Chapter Exercise</span>
<span style="text-align: left; color:green; font-family: 'Franklin Gothic Medium', sans-serif; margin-top: 0; margin-bottom: 0.5em; font-style: italic;"><big><font color='black'>Calculate The Payment Dates And Amounts For Five Bonds</big></font></span>




<span style="display:block;
    border-left: 12px solid green;
    font-family: 'Garamond', serif;
    line-height: 1.5;
    padding: 15px">
|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;October 9$^{th}$ 2028|&nbsp;&nbsp;&nbsp;1.50|
|&nbsp;&nbsp;&nbsp;December 31$^{st}$ 2028|&nbsp;&nbsp;&nbsp;5.00|
|&nbsp;&nbsp;&nbsp;November 28$^{th}$ 2030|&nbsp;&nbsp;&nbsp;3.50|
|&nbsp;&nbsp;&nbsp;October 10$^{th}$ 2027|&nbsp;&nbsp;&nbsp;2.50|
|&nbsp;&nbsp;&nbsp;February 18$^{th}$ 2029|&nbsp;&nbsp;&nbsp; 4.00|

</span>


 see [Chapter One Hints: Create a NumPy Array](https://patrickjhess.github.io/Hints-Results/Chapter_One_Hints.html#create-a-numpy-array), and check the [expected results here](https://patrickjhess.github.io/Hints-Results/Chapter_One_Results.html#create-a-numpy-array).

